# Redes Neurais - Projeto 4
# Modelos Recorrentes
---------------------
### Luis Filipe Menezes
#### RA: 164924

## 1. Objetivos:
Este caderno consiste na terceira entrega da disciplina de Redes Neurais realizada no programa de Pós Graduação em Ciência da Computação durante meu mestrado.

O projeto tem como objetivo:

- Implementar um modelo LSTM ou GRU para uma das tarefas abaixo:
1. Classificação de série temporais. O modelo deve receber uma janela temporal (qualquer tipo de dado) e classificar o conteúdo da janela.
2. Previsão. Treinar um modelo para predizer o valor de uma variável no instante t+k. O modelo deve receber os dados da série temporal (instantes anteriores a t – verificar tamanho da janela) e predizer um favor futuro. k a distância da predição. Por exemplo, podemos alimentar um modelo com dados de uma dada empresa (i.e. PETR3) e tentar predizer qual será o valor da ação daqui 5 dias (k==5)
3. Autoencoder recorrente. O modelo deve mapear a série temporal na própria série. O objetivo será avaliar como os dados estão representados no espaço latente

Para este projeto foi escolhido a tarefa 